### Read Videos & Convert to Frames

In [2]:
import os
import logging
import subprocess

In [3]:
logging.basicConfig(level=logging.INFO) 

In [4]:
def extractFrames(directory_path, output_directory):

    # list all files in the directory
    videos = [vid for vid in os.listdir(directory_path) if vid.lower().endswith('.mp4')]

    # process each file
    for vid in videos:
        video_path = os.path.join(directory_path, vid)
        output_path = os.path.join(output_directory, vid)       # ???????????????????????????????????? wtf

        # check if the folder already exists
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            logging.info(f"Folder '{vid}' created at {output_path}")
        else:
            logging.info(f"Folder '{vid}' already exists at {output_path}")
        
        ffmpeg_command = f"ffmpeg -i {video_path} {output_directory}/frame%03d.png"
        # what we want to name the files --> A_0001_rep01_frame001
        #                                    Dataset_Train/{participantName}_{sentence_ID}_rep{%02d}_frame%03d.png
        
        subprocess.run(ffmpeg_command, shell=True)

    return output_path

In [20]:
base_directory = 'Dataset_Train'

### Convert to Grayscale

In [6]:
from matplotlib import pyplot as plt
import cv2
import numpy as np

In [9]:
def convertToGrayScale(directory_path, output_directory):
    # List all frames of the video in the folder with the specified directory_path
    videoFrames = [frame for frame in os.listdir(directory_path)]       # name of file

    # List all 
    grayImagesList = [] 
    for frame in videoFrames:
        frame_path = os.path.join(directory_path, frame)
        grayImage = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)
        grayImagesList.append(grayImage.astype(np.int16))
        
        # Save the grayscale image to a directory
        output_path = os.path.join(output_directory, f"gray_{frame}")
        cv2.imwrite(output_path, grayImage)
        
    logging.info(f"video images '{output_directory[-10:]}' converted to grayscale and saved in '{output_directory}'")

    return grayImagesList

### Directional ATD

In [10]:
def performThreshold(diffFrame):
    mask = (diffFrame < 13) & (diffFrame > -13)
    diffFrame[mask] = 0
    return diffFrame

In [11]:
def binarize(noiseless_image):
    mean = np.mean(noiseless_image)
    maskZeroes = (noiseless_image < mean)
    maskOnes = (noiseless_image >= mean)
    binarizedImage = noiseless_image
    binarizedImage[maskZeroes] = 0
    binarizedImage[maskOnes] = 1
    return binarizedImage

In [12]:
def accumulateFrames(framesList):
    accumulatedFrame = framesList[0]

    for i in range(1, len(framesList)):
        accumulatedFrame += framesList[i]
    
    return accumulatedFrame

In [13]:
def applyForwardDiff(grayImagesList):
    listOfFrames = []

    for index in range(len(grayImagesList) - 1):
        forward_ImageDiff = grayImagesList[index] - grayImagesList[index + 1]
        forward_ImageDiff = performThreshold(forward_ImageDiff)
        binarizedImage = binarize(forward_ImageDiff)        
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

In [14]:
def applyBackwardDiff(grayImagesList):
    listOfFrames = []

    for index in range(1, len(grayImagesList)):
        backward_ImageDiff = grayImagesList[index] - grayImagesList[index - 1]
        backward_ImageDiff = performThreshold(backward_ImageDiff)
        binarizedImage = binarize(backward_ImageDiff) 
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

In [15]:
def applyBiDirectionalDiff(grayImagesList):
    listOfFrames = []

    for index in range(1, len(grayImagesList)-1):
        average_frame = np.mean([grayImagesList[index - 1], grayImagesList[index + 1]], axis=0)
        bidirectional_ImageDiff = grayImagesList[index] - average_frame
        bidirectional_ImageDiff = performThreshold(bidirectional_ImageDiff)
        binarizedImage = binarize(bidirectional_ImageDiff)
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

In [16]:
def applyATD(frames, output_directory, index):
    forwardATD = applyForwardDiff(frames)
    backwardATD = applyBackwardDiff(frames)
    bidirectATD = applyBiDirectionalDiff(frames)
    
    
    wordFolder = os.path.join(output_directory, f'word_{index}')

    # check if the folder already exists
    if not os.path.exists(wordFolder):
        os.makedirs(wordFolder)
        logging.info(f"Folder created at {wordFolder}")
    else:
        logging.info(f"Folder already exists at {wordFolder}")

    # Save the atd images (RGB) to a directory
    image_ForwardATD_path = os.path.join(wordFolder, f"forwardATD_word{index}.png")
    cv2.imwrite(image_ForwardATD_path, forwardATD)
    
    image_BackwardATD_path = os.path.join(wordFolder, f"backwardATD_word{index}.png")
    cv2.imwrite(image_BackwardATD_path, backwardATD)
    
    image_BidirectionalATD_path = os.path.join(wordFolder, f"bidirectionalATD_word{index}.png")
    cv2.imwrite(image_BidirectionalATD_path, bidirectATD)
        
    logging.info(f"ATD frames '{output_directory[-10:]}_word{index}' created and saved in '{wordFolder}'")

    stackedImage = np.dstack([forwardATD, backwardATD, bidirectATD]).astype(np.uint8)
    stackedImage_path = os.path.join(wordFolder, f"stackedImage_word{index}.png")
    cv2.imwrite(stackedImage_path, stackedImage)
    
    logging.info(f"Stacked Image '{output_directory[-10:]}_word{index}' created and saved in '{wordFolder}'")

### Main

#### _Creating External Files_

In [22]:
noWords = 2

In [21]:
## Image Sequence File ##
vidFrames_path = os.path.join('videoFrames')
if not os.path.exists(vidFrames_path):
    os.makedirs(vidFrames_path)
    logging.info(f"Folder 'videoFrames' created at {vidFrames_path}")

## Grayscale File ##
grayscaleFrames_path = os.path.join('grayscaleFrames')
if not os.path.exists(grayscaleFrames_path):
    os.makedirs(grayscaleFrames_path)
    logging.info(f"Folder 'grayscaleFrames' created at {grayscaleFrames_path}")
    
## ATD and Stacked Image ##
ATDStacked_path = os.path.join('ATD&Stacked')
if not os.path.exists(ATDStacked_path):
    os.makedirs(ATDStacked_path)
    logging.info(f"Folder 'ATDStacked' created at {ATDStacked_path}")

INFO:root:Folder 'videoFrames' created at videoFrames
INFO:root:Folder 'grayscaleFrames' created at grayscaleFrames
INFO:root:Folder 'ATDStacked' created at ATD&Stacked


#### _Iterating Through Files & Run:_
*   Image Sequences
*   Grayscaling
*   ATD & Image Stacking

In [19]:
# Iterate through the directory and its subdirectories
first_iteration = True
for root, dirs, files in os.walk(base_directory):
    
    # Skip the first iteration (base)
    if first_iteration:
        first_iteration = False
        continue  
    
    # Create folders for each participant (A, B, C, etc.) in each of the three folders (videoFrames, grayscaleFrames, ATDStackedFrames)
    participant_img_Path = os.path.join(vidFrames_path, os.path.relpath(root, base_directory))    
    if not os.path.exists(participant_img_Path):
        os.makedirs(participant_img_Path)
        
    participant_grayscale_Path = os.path.join(grayscaleFrames_path, os.path.relpath(root, base_directory))
    if not os.path.exists(participant_grayscale_Path):
        os.makedirs(participant_grayscale_Path)
        
    participant_atd_path = os.path.join(ATDStacked_path, os.path.relpath(root, base_directory))
    if not os.path.exists(participant_atd_path):
        os.makedirs(participant_atd_path)
    
    for directory in dirs:
        # Create folders for each sentence (1 --> 50) in each of the three folders (videoFrames, grayscaleFrames, ATDStackedFrames)
        sentenceID_img_path = os.path.join(participant_img_Path, f'_{directory}')
        if not os.path.exists(sentenceID_img_path):
            os.makedirs(sentenceID_img_path)
            
        sentenceID_grayscale_path = os.path.join(participant_grayscale_Path, f'_{directory}')
        if not os.path.exists(sentenceID_grayscale_path):
            os.makedirs(sentenceID_grayscale_path)
            
        sentenceID_atd_path = os.path.join(participant_atd_path, f'_{directory}')
        if not os.path.exists(sentenceID_atd_path):
            os.makedirs(sentenceID_atd_path)
        
        # Run Image Sequence Code
        vid_path = os.path.join(root, directory)
        output_path = extractFrames(vid_path, sentenceID_img_path)
        
        # Run GrayScale
        output_gray_path = os.path.join(sentenceID_grayscale_path, os.path.relpath(output_path, sentenceID_img_path))
        if not os.path.exists(output_gray_path):
            os.makedirs(output_gray_path)
        grayImagesList = convertToGrayScale(output_path, output_gray_path)
        
        # Run ATD
        output_atd_path = os.path.join(sentenceID_atd_path, os.path.relpath(output_gray_path, sentenceID_grayscale_path))
        # relpath --> compares output_gray_path & sentenceID_grayscale_path
        #             their difference is the actual sentence idea
        
        if not os.path.exists(output_atd_path):
            os.makedirs(output_atd_path)
        
        noFrames = len(grayImagesList)
        split = noFrames // noWords
        
        for i in range(noWords):
            index = split * i
            applyATD(grayImagesList[index: index + split], output_atd_path, i + 1)